# Prediction and Submission

This notebook loads a previously trained model, the test dataset and saves predictions compliant with the competition submission format.

In [ ]:
import os
from pathlib import Path

from biomasstry.datasets import TemporalSentinel2Dataset
from biomasstry.models import TemporalSentinel2Model
from PIL import Image
import torch
from tqdm.notebook import tqdm

In [ ]:
ARTIFACTS_DIR = Path("/notebooks/artifacts")
PREDICTIONS_DIR = Path("/notebooks/submission")
model_file = "20230102_TemporalS2_B16_E10.pt"
model_path = ARTIFACTS_DIR / model_file

## Test Data

In [ ]:
# Testing Dataset
testds = TemporalSentinel2Dataset(train=False)

## Pre-trained Model

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
model = TemporalSentinel2Model(n_samples=5, output_nc=1)
model.load_state_dict(torch.load(model_path))
model.eval()
model.to(device)

In [ ]:
def predict_agbm(inputs, model):
    pred = model.predict(inputs.unsqueeze(0))
    return pred.detach().squeeze().cpu().numpy()

In [ ]:
def save_agbm(agbm_pred, chipid):
    im = Image.fromarray(agbm_pred)
    save_path = os.path.join(PREDICTIONS_DIR, f'{chipid}_agbm.tif')
    im.save(save_path, format='TIFF', save_all=True)

In [ ]:
for ix, timg in tqdm(enumerate(testds), total=len(testds)):
    chipid = testds.chip_ids[ix]
    inputs = [img.to(device) for img in timg]
    agbm = predict_agbm(inputs, model)
    save_agbm(agbm, chipid)